# Lleva a cabo una Prueba Inicial de Validación del Modelo Entrenado viendo los objetos detectados en las imágenes de validación

0) Preparar ambiente e instalar paquetes:

In [ ]:
# nota se debe indicar la versión 1 de TF para compatibilidad del código
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

In [ ]:
#@title baja e instala los parquetes de 'Object Detection' de Tensor Flow a utilizar en el disco temporal de Colab (demora un ratito)

!pip install tf_slim

%cd /content
!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
model_dir = 'training/'

!python object_detection/builders/model_builder_test.py

os.environ['PYTHONPATH'] += ':/content/models/research/object_detection:/content/models/research/slim/object_detection'


1) Cargar librerías:

In [ ]:
import os
import os.path
import sys
import numpy as np

from IPython.display import Image, display
from PIL import Image as ImPIL

print ("Librerías cargadas.")

2) Montar el Drive:

In [ ]:
# monta Google Drive:
# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# configuración de directorios local en Google Drive
drive_path = '/content/gdrive/My Drive/GEMIS/objDetectionCursogramas'
data_dir_path = drive_path + '/Cursogramas'

print("Configuración de archivos definida")

3) Cargar el modelo entrenado:

In [ ]:
#@title carga el modelo de object detection entrenado

# path donde está el modelo exportado
ModelObjDetEntrenado = drive_path + '/TF_model/frozen_inference_graph.pb'

# archivo con lista de etiquetas para mostrar 
labelMapFile = data_dir_path + '/label_map.pbtxt'

# se debe ubicar en el directorio correspondiente
%cd /content/models/research/object_detection

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

# This is needed to display the images.
%matplotlib inline

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
 
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(ModelObjDetEntrenado, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(labelMapFile)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=90, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

# función auxiliar para ejecutar el modelo
def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict


print("Modelo objDetector cargado: [", ModelObjDetEntrenado, "], [", labelMapFile, "] ")


4) Llevar a cabo la detección de objetos sobre las imágenes de validación:

In [ ]:
# define la carpeta donde están las imágenes para procesar

#dirTest = data_dir_path + '/validation/images' 
#dirTestXML = data_dir_path + '/validation/annotations' 
dirTest = data_dir_path + '/Generados' 
dirTestXML = data_dir_path + '/Generados' 


# levanta las imágenes de prueba para procesar
process_FileNames = [ fn for fn in os.listdir( dirTest ) if fn.endswith('.png') or fn.endswith('.jpg') ]
print("> Imágenes a probar: ", len(process_FileNames))

In [ ]:
#@title Toma una muestra de las imágenes (si es necesario o se quiere) { run: "auto" }

porcMuestraImagenesProcesar = 100  #@param {type:"slider", min:0, max:100, step:5}
if porcMuestraImagenesProcesar>0 and porcMuestraImagenesProcesar<100:
  cantProcesar = int(len(process_FileNames)*porcMuestraImagenesProcesar/100)
  if cantProcesar==0:
    cantProcesar = 1
  process_FileNames = process_FileNames[:cantProcesar]
print("> Imágenes/XML a probar: ", len(process_FileNames))

In [ ]:
#@title Muestra los objetos detectados en las imágenes

# define minima probabilidad a usar
minimaProbabilidadObjectDetection = 95 #@param {type:"slider", min:1, max:100, step:1.0}
minProbObjDet = minimaProbabilidadObjectDetection / 100.0

# define si muestra detalle o no
muestraDetalleSubImagenes = False #@param {type:"boolean"}

# procesa las imágenes 
for fn in process_FileNames:

  # define archivo a procesar y generar
  imagenProcesar = dirTest + '/' + fn

  print("\n> ", imagenProcesar, ": ")
  
  # open file to process
  imageCargada = ImPIL.open(imagenProcesar) 
  imCargada_ancho, imCargada_alto = imageCargada.size

  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(imageCargada)

  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)

  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)

  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)

  # muestra la imagen con los objetos detectados
  display( ImPIL.fromarray(image_np, 'RGB') )

  if muestraDetalleSubImagenes:   

      # procesa los objetos detectados
      for detClass, detBox, detScore in zip(  output_dict['detection_classes'], output_dict['detection_boxes'], output_dict['detection_scores'] ):

          # si el objeto detectado tiene un puntaje superior o igual al mínimo
          if detScore>=minProbObjDet:

            # como las coordenadas están normalizadas las debe convertir 
            # teniendo en cuenta el tamaño de la imagen
            # además notar que vienen datas en otro orden
            # - detBox = (ini alto, ini ancho, fin alto, fin ancho)
            # - nuevoRangoIn = (ini ancho, ini alto, fin ancho, fin alto)    
            nuevoRangoIm = (detBox[1] * imCargada_ancho, 
                            detBox[0] * imCargada_alto,
                            detBox[3] * imCargada_ancho,
                            detBox[2] * imCargada_alto) 
            centroideIm = (nuevoRangoIm[2]-nuevoRangoIm[0]/2) 

            # extrae la subimage de acuerdo al área indicada por el detector        
            imDetObj = imageCargada.crop(nuevoRangoIm )
    
            # muestra la sub-imagen
            print("\n")
            display( imDetObj )

            # muestra resultados
            print("    - detecta ", detClass, " : ", detScore*100, "% : ", detBox, "con centroide: ", centroideIm)

